In [1]:
import cv2
import numpy as np

In [2]:
def calcularGauss(imagen, niveles)->list:
    copia = imagen.copy()
    piramide_gaussiana = [copia]
    for i in range(niveles):
        # Funcion de gauss
        copia = cv2.pyrDown(copia) #funcion de max gaus -> reshape
        piramide_gaussiana.append(copia)
    return piramide_gaussiana

def calcularLaplaciana(piramide_gaussiana):
    piramide_laplaciana = []
    for i in range(len(piramide_gaussiana)-1,0, -1):
        g_i = cv2.pyrUp(piramide_gaussiana[i]) #gi es la imagen incrementada en tamaño y la que deberia ser modificada si no hace match en tamaño
        print("i: ", i, "GI: ", g_i.shape, "GI-1", piramide_gaussiana[i-1].shape)
        g_i = ajustar_tamano(piramide_gaussiana[i-1], g_i)
        print("i: ", i, "GI: ", g_i.shape, "GI-1", piramide_gaussiana[i-1].shape)
        l_i = cv2.subtract(piramide_gaussiana[i-1], g_i)
        piramide_laplaciana.append(l_i)
    return piramide_laplaciana

def ajustar_tamano(img_norm, img_x2):
    canales = img_norm.shape[2]
    if img_x2.shape[0] < img_norm.shape[0]:
        empty_column = np.zeros((1, img_x2.shape[1], canales), dtype=np.uint8)
        img_x2 = np.vstack((img_norm, empty_column))
    elif img_x2.shape[0] > img_norm.shape[0]:
        # remove last row
        img_x2 = img_x2[:-1, :, :]
    
    if img_x2.shape[1] < img_norm.shape[1]:
        print("Entro Ajusto fila")
        empty_column = np.zeros((img_x2.shape[0], 1, canales), dtype=np.uint8)
        img_x2 = np.hstack((img_x2, empty_column))
        print(img_x2.shape)
    elif img_x2.shape[1] > img_norm.shape[1]:
        print("Entro Ajusto columna")
        img_x2 = img_x2[:, :-1, :]
    return img_x2

def reconstruir(laplacianas, gaussianas):
    img_r = gaussianas[-1]
    # img_r = cv2.pyrUp(gaussiana_i)
    for i in range(len(gaussianas)):
        print(i)
        try:
            laplaciana_i = laplacianas[-i-1]
        except:
            print("Error", i)
            break
        img_r = cv2.pyrUp(img_r)
        print("laplaciana_i", laplaciana_i.shape, "img_r", img_r.shape)
        img_r = ajustar_tamano(laplaciana_i, img_r)
        img_r = cv2.add(img_r, laplaciana_i)
        cv2.imwrite("img_r" + str(i) + ".jpg", img_r)


        

def guardar_imgs(laplacianas, gaussianas):
    for i in range(len(laplacianas)):
        cv2.imwrite("laplaciana" + str(i) + ".jpg", laplacianas[i])
    for i in range(len(gaussianas)):
        cv2.imwrite("gaussiana" + str(i) + ".jpg", gaussianas[i])

In [3]:
A = cv2.imread('woman.jpeg')
B = cv2.imread('girl.jpeg')
# convert to black and white
# A = cv2.cvtColor(A, cv2.COLOR_BGR2GRAY)
# B = cv2.cvtColor(B, cv2.COLOR_BGR2GRAY)
print("Shape A: ", A.shape, "Shape B: ", B.shape)
if A.shape[0]%2 ==1:
    A = A[:-1, :]
if A.shape[1]%2 ==1:
    A = A[:, :-1]
if B.shape[0]%2 ==1:
    B = B[:-1, :]
if B.shape[1]%2 ==1:
    B = B[:, :-1]
print("Shape A: ", A.shape, "Shape B: ", B.shape)

ga = calcularGauss(A, 3)
gb = calcularGauss(B, 3)

la = calcularLaplaciana(ga)
lb = calcularLaplaciana(gb)

Shape A:  (338, 282, 3) Shape B:  (338, 282, 3)
Shape A:  (338, 282, 3) Shape B:  (338, 282, 3)
i:  3 GI:  (86, 72, 3) GI-1 (85, 71, 3)
Entro Ajusto columna
i:  3 GI:  (85, 71, 3) GI-1 (85, 71, 3)
i:  2 GI:  (170, 142, 3) GI-1 (169, 141, 3)
Entro Ajusto columna
i:  2 GI:  (169, 141, 3) GI-1 (169, 141, 3)
i:  1 GI:  (338, 282, 3) GI-1 (338, 282, 3)
i:  1 GI:  (338, 282, 3) GI-1 (338, 282, 3)
i:  3 GI:  (86, 72, 3) GI-1 (85, 71, 3)
Entro Ajusto columna
i:  3 GI:  (85, 71, 3) GI-1 (85, 71, 3)
i:  2 GI:  (170, 142, 3) GI-1 (169, 141, 3)
Entro Ajusto columna
i:  2 GI:  (169, 141, 3) GI-1 (169, 141, 3)
i:  1 GI:  (338, 282, 3) GI-1 (338, 282, 3)
i:  1 GI:  (338, 282, 3) GI-1 (338, 282, 3)


In [4]:
for i in ga:
    print(i.shape)
print("Laplace")
for i in la:
    print(i.shape)

(338, 282, 3)
(169, 141, 3)
(85, 71, 3)
(43, 36, 3)
Laplace
(85, 71, 3)
(169, 141, 3)
(338, 282, 3)


In [5]:
print("Gaussiana A: ", len(ga), "Gaussiana B: ", len(gb))
print("Laplaciana A: ", len(la), "Laplaciana B: ", len(lb))
print("Gaussiana A size: ", ga[-1].shape, "Gaussiana B size: ", gb[-1].shape)
print("Laplaciana A size: ", la[-1].shape, "Laplaciana B size: ", lb[-1].shape)



Gaussiana A:  4 Gaussiana B:  4
Laplaciana A:  3 Laplaciana B:  3
Gaussiana A size:  (43, 36, 3) Gaussiana B size:  (43, 36, 3)
Laplaciana A size:  (338, 282, 3) Laplaciana B size:  (338, 282, 3)


In [6]:
reconstruir(la[::-1], gb)

0
laplaciana_i (85, 71, 3) img_r (86, 72, 3)
Entro Ajusto columna
1
laplaciana_i (169, 141, 3) img_r (170, 142, 3)
Entro Ajusto columna
2
laplaciana_i (338, 282, 3) img_r (338, 282, 3)
3
Error 3


In [70]:
len(la)

2

In [ ]:
len(ga)